https://huggingface.co/datasets/FreedomIntelligence/Huatuo26M-Lite

https://hf-mirror.com/datasets/FreedomIntelligence/Huatuo26M-Lite

In [1]:
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import json

In [2]:
# 数据集
path = "FreedomIntelligence/Huatuo26M-Lite" # hugginface 地址
path = "./Huatuo26M-Lite"                   # 本地路径
# 保存路径
xtuner_data_path = "./Huatuo26M-Lite-xtuner-data.json"

In [3]:
dataset = load_dataset(path)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'answer', 'score', 'label', 'question', 'related_diseases'],
        num_rows: 177703
    })
})

In [4]:
dataset["train"][0]

{'id': 22647835,
 'answer': '治疗鼻中隔偏曲的方法有手术和非手术治疗两种，手术治疗是通过手术矫正鼻中隔偏曲，非手术治疗则是通过药物治疗和物理治疗来缓解症状。手术治疗是治疗鼻中隔偏曲的最有效方法，手术后需要注意休息，避免剧烈运动和低头工作，同时也要注意饮食，少吃辛辣食物和不喝酒。手术后两周内鼻涕或痰中出现血水或血块是正常现象，若出现大量出血、发烧、剧烈疼痛时请尽速就医。',
 'score': 5,
 'label': '眼耳鼻喉科',
 'question': '上个月感冒了，也没有用药，感冒好了以后就觉得鼻子经常不通畅，鼻子还经常晦气红皮、发痒、而且还会有头晕，一直都以为是上次感冒留下的后遗症，去医院检查，检查结果出来以后说是鼻中隔偏曲。请问如何治疗鼻中隔偏曲？',
 'related_diseases': '鼻中隔偏曲'}

In [20]:
system = "你现在是一名医生，具备丰富的医学知识和临床经验。你擅长诊断和治疗各种疾病，能为病人提供专业的医疗建议。你有良好的沟通技巧，能与病人和他们的家人建立信任关系。请在这个角色下为我解答以下问题。"

In [24]:
xtuner_format = {
    "conversation": [
        {
            "system": "",
            "input": "",
            "output": ""
        }
    ]
}

In [25]:
xtuner_dataset = []
for data in tqdm(dataset["train"]):
    if data['answer'].strip() == "" or data["question"].strip() == "":
        print("empty data")
        continue
    xtuner_format_c = deepcopy(xtuner_format)
    xtuner_format_c["conversation"][0]["system"] = system
    xtuner_format_c["conversation"][0]["input"] = data["question"]
    xtuner_format_c["conversation"][0]["output"] = data["answer"]
    xtuner_dataset.append(xtuner_format_c)

100%|██████████| 177703/177703 [00:13<00:00, 12878.03it/s]


In [29]:
xtuner_dataset[0]

{'conversation': [{'system': '你现在是一名医生，具备丰富的医学知识和临床经验。你擅长诊断和治疗各种疾病，能为病人提供专业的医疗建议。你有良好的沟通技巧，能与病人和他们的家人建立信任关系。请在这个角色下为我解答以下问题。',
   'input': '上个月感冒了，也没有用药，感冒好了以后就觉得鼻子经常不通畅，鼻子还经常晦气红皮、发痒、而且还会有头晕，一直都以为是上次感冒留下的后遗症，去医院检查，检查结果出来以后说是鼻中隔偏曲。请问如何治疗鼻中隔偏曲？',
   'output': '治疗鼻中隔偏曲的方法有手术和非手术治疗两种，手术治疗是通过手术矫正鼻中隔偏曲，非手术治疗则是通过药物治疗和物理治疗来缓解症状。手术治疗是治疗鼻中隔偏曲的最有效方法，手术后需要注意休息，避免剧烈运动和低头工作，同时也要注意饮食，少吃辛辣食物和不喝酒。手术后两周内鼻涕或痰中出现血水或血块是正常现象，若出现大量出血、发烧、剧烈疼痛时请尽速就医。'}]}

In [26]:
with open(xtuner_data_path, "w", encoding="utf-8") as f:
    json.dump(xtuner_dataset, f, ensure_ascii=False, indent=4)